# Brain Tumor Detection
Description
This dataset was originally created by Yousef Ghanem. To see the current project, which may have been updated since this version, please go here: https://universe.roboflow.com/yousef-ghanem-jzj4y/brain-tumor-detection-fpf1f.

This dataset is part of RF100, an Intel-sponsored initiative to create a new object detection benchmark for model generalizability.

Access the RF100 Github repo: https://github.com/roboflow-ai/roboflow-100-benchmark

## Imports

In [ ]:
# Go to project root folder
import os
os.chdir("../")
%pwd

In [ ]:
from tqdm.notebook import tqdm

from dotenv import load_dotenv
load_dotenv()

import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
tf.config.list_physical_devices('GPU'), tf.__version__

In [ ]:
# auto reload dotenv 
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

In [ ]:
from src.utils.visualization_funcs import plot_random_images_bbox

## Paths Setup

In [ ]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

In [ ]:
cfg.DATASET_DIRS

In [ ]:
from pathlib import Path

DATASET_DIRS = Path(cfg.DATASET.DATASET_DIR)
TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)

MODEL_CHECKPOINT = Path(cfg.OUTPUTS.CHECKPOINT_PATH)

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE

CLASS_NAME = [
    'label0',
    'label1',
    'label2'
]
class_map = {v: k for k, v in enumerate(CLASS_NAME)}
class_map[CLASS_NAME[0]]

## Dataset Download from Roboflow

In [ ]:
if not TRAIN_DIR.exists():
    from roboflow import Roboflow
    rf = Roboflow()
    # https://universe.roboflow.com/roboflow-100/brain-tumor-m2pbp/dataset/2/images?split=test
    project = rf.workspace("roboflow-100").project("brain-tumor-m2pbp")
    version = project.version(2)
    dataset = version.download("tensorflow")

## Load images from directory

In [ ]:
from src.data_handler.annotation_processor import AnnotationProcessor
from src.data_handler.data_loader import DataLoader
from src.data_handler.preprocessor import Preprocessor
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
prepare_test_dataset = AnnotationProcessor(annotation_file= str(TEST_DIR/'_annotations.csv'))
test_image_paths, test_class_ids, test_bboxes  = prepare_test_dataset.process_annotations(image_dir=TEST_DIR, class_id_map=_class_map)
test_dl = DataLoader(test_image_paths, test_class_ids, test_bboxes).load_val_dataset()
test_ds = Preprocessor(test_dl).preprocess()
test_ds = test_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

## Load the model

In [ ]:
import keras
# model = keras.models.load_model(str(MODEL_CHECKPOINT/'chpt_49.keras'))
# model = keras.models.load_model('output/checkpoints/ckpt_49.keras')

In [ ]:
import mlflow
model_uri = 'runs:/64794e1f5e564397b73c5fa271fec996/model'

# This is the input example logged with the model
model =mlflow.tensorflow.load_model(model_uri)
input_data = model.input_example

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
# mlflow.models.predict(
#     model_uri=model_uri,
#     input_data=input_data,
#     env_manager="uv",
# )

In [ ]:
results = model.evaluate(test_ds, return_dict=True)
results

In [ ]:
unnorm_bbx = []
for bbx in test_bboxes:
   unnorm_bbx.append(bbx*IMG_SIZE)

In [ ]:
import random


def plot_random_images_bbox(*,  random_samples:np.ndarray, image_paths:np.ndarray, class_ids:np.ndarray, bboxes:np.ndarray, class_map:dict) -> None:
  fig = plt.figure(figsize=(8, 8))

  print(f"Random samples: {random_samples}")
  class_map_invert = {v: k for k, v in class_map.items()}
  
  for i, idx in enumerate(random_samples):
    ax = fig.add_subplot(3, 3, i+1)
    image = image_paths[idx]
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Create title from class IDs
    title_labels = [class_map_invert[int(cls_id)] for cls_id in class_ids[idx]]
    title = ", ".join(title_labels)
    ax.set_title(title)
    ax.imshow(image) #display image before bounding box

    # Draw bounding boxes with different colors
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0,255,255), (255,0,255)] # Example colors
    for j, (xmin, ymin, xmax, ymax) in enumerate(bboxes[idx]):
        color = colors[j % len(colors)] # Cycle through colors
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 1)
    ax.imshow(image) #display image with bounding box.

  plt.tight_layout() #prevents overlapping subplots
  plt.show()

In [ ]:
random_samples = random.sample(range(len(test_images)), 9)
plot_random_images_bbox(random_samples=random_samples,
                        image_paths=test_images, 
                        class_ids=test_class_ids, 
                        bboxes=unnorm_bbx,
                        class_map=class_map)

In [ ]:
pred_cls_id, pred_bbx = model.predict(test_ds)
print(pred_cls_id.shape, pred_bbx.shape)

In [ ]:
pred_bbx

In [ ]:
unnorm_pred_bbx = []
for bbx in pred_bbx:
   unnorm_pred_bbx.append(bbx*IMG_SIZE)

In [ ]:
pred_cls_id = (pred_cls_id>0.5).astype(int)
pred_cls_id

In [ ]:
pred_cls = [np.where(row==1)[0].tolist() for row in pred_cls_id]
pred_cls

In [ ]:
for idx in random_samples:
   print([cls_id for cls_id in pred_cls[idx]])

In [ ]:
plot_random_images_bbox(random_samples=random_samples,
                        image_paths=test_images, 
                        class_ids=pred_cls, 
                        bboxes=unnorm_pred_bbx,
                        class_map=class_map)

In [ ]:
unnorm_bbx, pred_bbx

In [ ]:
from src.losses.iou_loss import iou_metric
for y_tbbx, y_prdbbx in zip(unnorm_bbx, unnorm_pred_bbx):
    print(iou_metric(y_true=y_tbbx, y_pred=y_prdbbx)) 